# トリプルをNeo4jへ流し込んでみる

Phase2で作ったトリプルをNeo4jに流し込む。事前にNeo4jはdockerで立ち上げておく。

## 事前準備

Neo4jを立ち上げる

```bash
docker compose up -d
```

ブラウザで [http://localhost:7474/]( http://localhost:7474/) でWebインターフェースに入れる。


## ライブラリ、環境変数

In [1]:
import json
import os
from pathlib import Path

from dotenv import load_dotenv
from neo4j import GraphDatabase

In [2]:
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "")

print(f"URI: {NEO4J_URI}")
print(f"USER: {NEO4J_USER}")

URI: bolt://localhost:7687
USER: neo4j


## データ

In [3]:
def load_triples_from_file(path: Path) -> list[dict]:
    with path.open(encoding="utf-8") as f:
        return json.load(f)

In [4]:
# 別で出力してあるトリプル（JSONファイル）を読み込む
TRIPLES_JSON = Path("../data/phase_2_outputs/triples_llm.json")

triples = load_triples_from_file(TRIPLES_JSON)
print(len(triples))

6


In [5]:
triples[0]

{'subject': 'logistic_regression', 'verb': 'be', 'object': 'model'}

## Neo4jにデータをinsertする

In [6]:
def insert_triples(triples: list[dict]) -> None:
    """List[dict] のトリプルを Neo4j に投入する"""

    driver = GraphDatabase.driver(
        NEO4J_URI,
        auth=(NEO4J_USER, NEO4J_PASSWORD)
    )

    query = """
    UNWIND $rows AS row
    // ノード（エンティティ）を作成/再利用
    MERGE (s:Entity {name: row.subject})
    MERGE (o:Entity {name: row.object})

    // 関係を作成（REL というラベルを付け、verb をプロパティに持たせる）
    MERGE (s)-[r:REL {verb: row.verb}]->(o)

    // もし追加のプロパティ（source_text など）があれば、ここで SET する
    // 例:
    //   SET r.source_text = row.source_text
    """

    with driver.session() as session:
        session.run(query, rows=triples)

    driver.close()

In [7]:
insert_triples(triples)

Neo4j のWebインターフェースでグラフを確認できる